In [51]:
import random
import numpy as np
import networkx as nx 
from dijkstar import Graph, find_path

In [52]:
def generate_instance_matrix(nb_vertex, min_weight, max_weight, density):
  graph = np.ones((nb_vertex, nb_vertex)) * np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < density:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i,line in enumerate(graph):
    index_zeros = np.where(line == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_nodes(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, node_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(node_list), len(node_list)))
  for i in node_list:
    voisins_array = np.delete(node_list, np.where(node_list == i))
    for j in voisins_array:
      distance, path = get_distance_between_nodes(converted_graph, i, j)
      converted_graph_i = np.where(node_list == i)[0][0]
      converted_graph_j = np.where(node_list == j)[0][0]
      complete_graph[converted_graph_i][converted_graph_j] = distance
      all_paths[(converted_graph_i, converted_graph_j)] = path
  return complete_graph, all_paths

def get_random_vertex(size, nb_vertex):
  random_vertex = []
  all_vertex = np.array(range(size))
  for _ in range(nb_vertex):
    value = random.choice(all_vertex)
    random_vertex.append(value)
    all_vertex = np.delete(all_vertex, np.where(all_vertex == value))
  return np.array(random_vertex)

def generate_interval():
  interval = random.choice(range(1,4))
  shift = random.choice(range(0, 11-interval))
  return(shift*60, (shift+interval)*60)

def generate_instance(nb_vertex_matrix, min_weight, max_weight, density, random_vertex):
  graph = generate_instance_matrix(nb_vertex_matrix, min_weight, max_weight, density)
  complete_graph, all_paths = get_complete_graph(graph, random_vertex)
  return graph, complete_graph, all_paths

def get_complete_path(path, all_paths):
  complete_path = []
  for i in range(len(path) - 1):
    complete_path.extend(all_paths[(path[i], path[i+1])][:-1])
    if i == len(path) - 2:
      complete_path.append(all_paths[(path[i], path[i+1])][-1])
  return complete_path

def is_path_in_interval(path, complete_graph, interval, vertex):
  weight = 0
  for i in range(np.where(np.array(path) == vertex)[0][0]):
    weight += complete_graph[path[i]][path[i+1]]
  if weight >= interval[0] and weight <= interval[1]:
    return True
  else:
    return False

def generate_neighbours(path, complete_graph, interval, vertex):
  neighbours = []
  for i in range(len(path)):
    for j in range(i + 1, len(path)):
      new_neighbour = path.copy()
      new_neighbour[i] = path[j]
      new_neighbour[j] = path[i]
      if is_path_in_interval(new_neighbour, complete_graph, interval, vertex):
        neighbours.append(new_neighbour)
  return neighbours

def generate_random_path(nb_vertex, start_vertex):
  path = np.arange(nb_vertex)
  path = np.delete(path, np.where(path == start_vertex))
  np.random.shuffle(path)
  path = np.append(path, start_vertex)
  path = np.insert(path, 0, start_vertex)
  return path

In [53]:
# variables
nb_vertex_matrix = 100
min_weight = 10
max_weight = 60
density = 0.3
nb_vertex_chosen = 4

In [54]:
#random generation of a list of vertex among all the vertex available
random_vertex = get_random_vertex(nb_vertex_matrix, nb_vertex_chosen)

# generation of the graph, the complete graph and the list of all paths
graph, complete_graph, all_paths =\
generate_instance(
  nb_vertex_matrix = nb_vertex_matrix,
  min_weight = min_weight,
  max_weight = max_weight,
  random_vertex = random_vertex,
  density = density
)

In [55]:
#generate an interval instance
interval = generate_interval()

In [56]:
#generate a random path
path = generate_random_path(nb_vertex_chosen,0)

In [57]:
# generate the neighbours of a path
neighbours = generate_neighbours(path, complete_graph, (0,50), 0)

In [58]:
# print("graph")
# print(graph)
print("complete_graph")
print(complete_graph)
# print("all_paths")
# print(all_paths)
# print("random_vertex")
# print(random_vertex)
# print("interval")
# print(interval)

complete_graph
[[ 0. 38. 35. 46.]
 [38.  0. 37. 23.]
 [35. 37.  0. 43.]
 [46. 23. 43.  0.]]


In [59]:
complete_path = get_complete_path(path, all_paths)

In [60]:
print(random_vertex)
print(path)
print(complete_path)


[ 7  8 79  9]
[0 2 3 1 0]
[7, 79, 32, 83, 9, 96, 8, 7]


In [121]:
from pulp import *
import numpy as np


X=[]
P = []
X_dict = {}


# variables du problème
for i in range(len(complete_graph)):
    for j in range(len(complete_graph)):
        X.append(LpVariable("X_" + str(i) + "_" + str(j), 0, 1, LpInteger))
        X_dict[("X_" + str(i) + "_" + str(j))] = LpVariable("X_" + str(i) + "_" + str(j), 0, 1, LpInteger)
        P.append(complete_graph[i][j])

# probleme
prob = LpProblem("plus court chemin", LpMinimize)

# objectif
# Somme de la multiplication des poids de chaque arc
prob += lpSum([P[i] * X[i] for i in range(len(P))])

# contraintes
# La somme d'un arc doit etre egale a 1
X_np = np.array(X)
X_np_reshape = X_np.reshape(len(complete_graph), len(complete_graph))

for i in range(len(X_np_reshape)):

    prob += lpSum(X_np_reshape[i]) == 1
    prob += X_np_reshape[i][i] == 0

    for j in range(len(X_np_reshape)):
        if i != j:
            prob += X_np_reshape[i][j] + X_np_reshape[j][i] <= 1

prob.solve()
print(LpStatus[prob.status])
print("Min=", value(prob.objective))

# variables resultat
for v in prob.variables():
    print("%s=%.2f"%(v.name,v.varValue), end=', ')


Optimal
Min= 133.0
X_0_0=0.00, X_0_1=1.00, X_0_2=0.00, X_0_3=0.00, X_1_0=0.00, X_1_1=0.00, X_1_2=1.00, X_1_3=0.00, X_2_0=1.00, X_2_1=0.00, X_2_2=0.00, X_2_3=0.00, X_3_0=0.00, X_3_1=1.00, X_3_2=0.00, X_3_3=0.00, 

In [62]:
nb_iterations=10000 #Dans la boucle while de la boucle principale
T0 = 10000.0 #Température de départ
Tmin = 1e-2 #Température minimale
tau = 1e4 #Pour la loi de refroidissement

In [63]:
def distance_tot(matrice,path_matrix):
    distance_tot = 0
    ind = 2
    points = [0,0]
    for i in range(len(path_matrix)):
        if i == 0:
            points = path_matrix[0:2]
            distance_tot += get_distance_between_nodes(matrice,points[0],points[1])[0]
        else:
            if (ind <= len(matrice)):
                points = path_matrix[ind-1:ind+1]
                distance_tot += get_distance_between_nodes(matrice,points[0],points[1])[0]
                ind += 1
    return distance_tot

print(distance_tot(complete_graph,path))


139.0


In [64]:
def recuit_simule_matrix(matrix,path):

    distance = distance_tot(matrix,path)
    solution = np.copy(path)
    i = 0
    T=T0

    while T>Tmin and i<nb_iterations:
        i+=1

        #Loi de refroidissement
        T = T0 * np.exp(-i/tau)

        #Création de la nouvelle solution proche de la precedente
        solution_voisine = neighbours[np.random.randint(0, len(neighbours))]

        #Comparaison des deux valeurs
        difference = distance_tot(matrix,solution) - distance_tot(matrix,solution_voisine)

        # Si la nouvelle solution est meilleure, on la prend
        if difference > 0:
            solution = solution_voisine
            distance = distance_tot(matrix,solution_voisine)
        # Sinon, on prend la solution actuelle
        else:
            if np.random.uniform() > np.exp(-difference/T): #on accepte la solution avec une certaine probabilité. 
                solution=solution_voisine
                distance=distance_tot(matrix,solution_voisine)
        
    return solution,distance
    
print(recuit_simule_matrix(complete_graph,path))


(array([0, 0, 3, 1, 2]), 106.0)
